##### Testing to directly extract data from pdf , which did not gived good result

In [6]:
import pdfplumber
import pandas as pd

# Path to the PDF file
pdf_file_path = "sample.pdf"

# List to hold extracted data
data = []

# Function to process text blocks
def process_text_block(text_block):
    lines = text_block.strip().split("\n")
    if len(lines) < 8:
        return None
    try:
        part_sno = int(lines[0].split(":")[1].strip())
        voter_full_name = lines[1].split(":")[1].strip()
        relative_name = lines[2].split(":")[1].strip()
        relation_type = "HSBN" if "Husband" in lines[2] else "FTHR"
        age = int(lines[3].split(":")[1].strip())
        gender = lines[4].split(":")[1].strip()
        house_no = lines[5].split(":")[1].strip()
        epic_no = lines[6].split(":")[1].strip()
    except (IndexError, ValueError):
        return None

    return [part_sno, voter_full_name, relative_name, relation_type, age, gender, house_no, epic_no]

# Open the PDF file
with pdfplumber.open(pdf_file_path) as pdf:
    # Iterate through pages
    for page in pdf.pages:
        text = page.extract_text()

        # Split text into blocks based on the pattern in the PDF (adjust as necessary)
        text_blocks = text.split("Photo Available")
        
        # Process each block
        for block in text_blocks:
            extracted_data = process_text_block(block)
            if extracted_data:
                data.append(extracted_data)

# Convert the extracted data into a DataFrame
df = pd.DataFrame(data, columns=["Part S.No", "Voter Full Name", "Relative's Name", "Relation Type", "Age", "Gender", "House No", "EPIC No"])

# Save the DataFrame to an Excel file
output_file = "extracted_data.xlsx"
df.to_excel(output_file, index=False)

print(f"Data successfully extracted and saved to {output_file}")


Data successfully extracted and saved to extracted_data.xlsx


In [7]:
df.head()

,Part S.No,Voter Full Name,Relative's Name,Relation Type,Age,Gender,House No,EPIC No


## Converting PDF Pages to Images

In [70]:
from pdf2image import convert_from_path

# Path to the PDF file
pdf_file_path = "sample.pdf"

# Convert PDF to a list of images (one image per page)
images = convert_from_path(pdf_file_path, dpi=300)  # Adjust DPI as needed

# Save images to a directory (optional)
image_paths = []
for i, image in enumerate(images):
    image_path = f"data/page_{i + 1}.png"
    image.save(image_path, "PNG")
    image_paths.append(image_path)

print("PDF pages converted to images.")


PDF pages converted to images.


In [9]:
import pytesseract
from PIL import Image
import pandas as pd

# Function to extract text from image
def extract_text(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# Function to parse text and create dataframe
def parse_text(text):
    # Split text into blocks
    blocks = text.split('\n\n')
    
    data = []
    for block in blocks:
        print(block)
        lines = block.split('\n')
        if len(lines) >= 4:
            part_no = lines[0].split()[0]
            name = lines[1].split(':')[1].strip()
            relative_name = lines[2].split(':')[1].strip()
            relation_type = 'HSBN' if 'Husband' in lines[2] else 'FTHR'
            age_gender = lines[3].split(':')[1].strip()
            age = age_gender.split()[0]
            gender = 'M' if 'Male' in age_gender else 'F'
            epic_no = lines[-1].strip()
            
            data.append([part_no, name, relative_name, relation_type, age, gender, '', epic_no])
    
    return pd.DataFrame(data, columns=['Part S.No', 'Voter Full Name', 'Relative\'s Name', 'Relation Type', 'Age', 'Gender', 'House No', 'EPIC No'])

# Main process
image_path = 'data/page_1.png'
extracted_text = extract_text(image_path)
df = parse_text(extracted_text)

# Save to Excel
df.to_excel('voter_data.xlsx', index=False)

Assembly Constituency No and Name : 260-KARAD SOUTH
Section No and Name: 1-Kalika Devi Mendir Parisar Ravivar Pet Karad Ishany 258
a)
Name : Lina Shaktikumar Chavan
Husband's Name: Shaktikumar Chavan
House Number :
Age : 37 Gender : Female
[4]
Name : siddharth anil mahajan
Father's Name: anil mahajan
House Number :
Age : 30 Gender : Male
Name : vaishali hanmant pawar
Husband's Name: hanmant pawar
House Number :
Age : 40 Gender + Female
Name : Ajit Pralhad Yadav
Father's Name: Pralhad Yadav
House Number :
Age : 33 Gender : Male
Name : jakiya talyabali mulla
Father's Name: taiyabali mulla
House Number : -
Age : 24 Gender : Female
Name : Swarupa prasad sankapal
Father's Name: prasad sankapal
House Number ; -
Age : 23 Gender : Female
Name : RAJASHRI KASAR
Others: VASANT KASAR
House Number : 0
Age : 48 Gender : Female
[|
Name : Padmavati Mukundas Laddad
Husband's Name: Mukundas Laddad
House Number : 6a
Age : 76 Gender : Female
ae)
Name : Pritamm Mukudadas Laddad
Father's Name: Mukudadas Lad

# Main Code

#### Importing all necessary libraries

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
import pandas as pd
import re
from scipy.ndimage import sobel

### Function to sort contours in a particular order (left-to-right, top-to-bottom, etc.)

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    return (cnts, boundingBoxes)

### Function to detect blocks in the image

In [ ]:
def get_blocks(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Improved preprocessing with GaussianBlur and thresholding
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Dilation to emphasize the text
    kernel = np.ones((3,3),np.uint8)
    thresh = cv2.dilate(thresh, kernel, iterations=1)

    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    (cnts, _) = sort_contours(cnts, method="top-to-bottom")

    blocks = []
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        if w > 100 and h > 100:  # Adjust these values based on your image
            blocks.append((x, y, x+w, y+h))

    blocks.sort(key=lambda b: (b[1] // 100, b[0]))  # Sort by row then column
    return blocks, image

### Function to extract the EPIC number from a specific region in the block

In [ ]:
def extract_epic_no(image, block, block_text):
    x1, y1, x2, y2 = block
    
    # Focus on the region where EPIC number is typically located
    epic_region = image[y1:y1+100, x2-300:x2]  # Adjust these values as per your image structure
    
    # Preprocess the image for better OCR
    gray = cv2.cvtColor(epic_region, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (1, 1), 0)
    sharpen = cv2.addWeighted(gray, 1.5, gray, -0.5, 0)
    denoised = cv2.fastNlMeansDenoising(sharpen)
    thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    # Use psm 7 for text line detection
    custom_config = r'--oem 3 --psm 7'
    text = pytesseract.image_to_string(thresh, config=custom_config)
    
    # Improved EPIC No extraction with correction
    epic_no = ''
    epic_pattern = r'[A-Z]{3}\d{6,}'
    match = re.search(epic_pattern, text)
    if match:
        epic_no = match.group()
    
    # If the epic number in the expected region is not found, search in the block text
    if not epic_no:
        match = re.search(epic_pattern, block_text)
        if match:
            epic_no = match.group()
    
    return epic_no

### Function to check for "DELETED" watermark in the block using gradients

In [ ]:
def check_deleted_watermark(image, block):
    x1, y1, x2, y2 = block
    roi = image[y1:y2, x1:x2]
    
    # Convert ROI to grayscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Use psm 6 for block of text detection
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(thresh, config=custom_config)
    
    # Check if the "DELETED" watermark is present in the block text
    if "DELETED" in text.upper():
        return True
    
    return False

### Function to extract text from the identified block

In [ ]:
def extract_text_from_block(image, block):
    x1, y1, x2, y2 = block
    roi = image[y1:y2, x1:x2]
    
    # Preprocess the image for better OCR
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (1, 1), 0)
    sharpen = cv2.addWeighted(gray, 1.5, gray, -0.5, 0)
    denoised = cv2.fastNlMeansDenoising(sharpen)
    thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    # Use psm 6 for block of text
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(thresh, config=custom_config)
    
    return text

### Function to extract the EPIC number from a specific region in the block

In [ ]:
def extract_epic_no(image, block, block_text):
    x1, y1, x2, y2 = block
    
    # Focus on the region where EPIC number is typically located
    epic_region = image[y1:y1+100, x2-300:x2]  # Adjust these values as per your image structure
    
    # Preprocess the image for better OCR
    gray = cv2.cvtColor(epic_region, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (1, 1), 0)
    sharpen = cv2.addWeighted(gray, 1.5, gray, -0.5, 0)
    denoised = cv2.fastNlMeansDenoising(sharpen)
    thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    # Use psm 7 for text line detection
    custom_config = r'--oem 3 --psm 7'
    text = pytesseract.image_to_string(thresh, config=custom_config)
    
    # Improved EPIC No extraction with correction
    epic_no = ''
    epic_pattern = r'[A-Z]{3}\d{6,}'
    match = re.search(epic_pattern, text)
    if match:
        epic_no = match.group()
    
    # If the epic number in the expected region is not found, search in the block text
    if not epic_no:
        match = re.search(epic_pattern, block_text)
        if match:
            epic_no = match.group()
    
    return epic_no

### Function to parse and clean the extracted text

In [ ]:
def parse_block_text(text, epic_no, counter):
    # Remove "Available" and "Photo" keywords
    text = re.sub(r'\b(Available|Photo)\b', '', text, flags=re.IGNORECASE)
    
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    data = {}
    for line in lines:
        if ':' in line:
            key, value = line.split(':', 1)
            data[key.strip()] = value.strip()
    
    name = data.get('Name', '')
    relative_name = data.get("Father's Name", '') or data.get("Husband's Name", '')
    relation_type = 'HSBN' if "Husband's Name" in data else 'FTHR'
    
    # Improved age and gender extraction
    age_gender = data.get('Age', '')
    age_match = re.search(r'(\d+)\s*\+?\s*', age_gender)
    age = age_match.group(1) if age_match else ''
    
    gender = 'M' if 'Male' in age_gender else 'F'
    
    # Improved house number extraction
    house_no = data.get('House Number', '').split()[0] if data.get('House Number') else ''
    
    return [counter, name, relative_name, relation_type, age, gender, house_no, epic_no]

## Final Computation

In [78]:

# Main processing function
def process_image(image_path):
    blocks, image = get_blocks(image_path)
    data = []
    counter = 1
    for block in blocks:
        # Check if the block contains the "DELETED" watermark
        if check_deleted_watermark(image, block):
            print(f"Block {counter} skipped due to DELETED watermark.")
            continue
        
        block_text = extract_text_from_block(image, block)
        epic_no = extract_epic_no(image, block, block_text)
        
        # Ensure that EPIC No detected is in the correct format and location
        if len(epic_no) == 0:
            # In epic number store block_text part before name
            epic_no = block_text.split("Name")[0].strip()
            # Remove extra white space in between
            epic_no = " ".join(epic_no.split())
        
        # print(f"Block {counter} Text: {block_text}")
        # print(f"Block {counter} EPIC No: {epic_no}")
        
        parsed_data = parse_block_text(block_text, epic_no, counter)
        if parsed_data[1]:  # Only add if name is not empty
            data.append(parsed_data)
            counter += 1
    
    df = pd.DataFrame(data, columns=['S.No', 'Voter Full Name', 'Relative\'s Name', 'Relation Type', 'Age', 'Gender', 'House No', 'EPIC No'])
    return df


# Main process to handle multiple images
def process_multiple_images(image_paths):
    combined_data = []
    counter = 1
    for image_path in image_paths:
        # print(f"Processing {image_path}...")
        blocks, image = get_blocks(image_path)
        for block in blocks:
            # Check if the block contains the "DELETED" watermark
            if check_deleted_watermark(image, block):
                print(f"Block {counter} skipped due to DELETED watermark.")
                continue
            
            block_text = extract_text_from_block(image, block)
            epic_no = extract_epic_no(image, block, block_text)
            
            # Ensure that EPIC No detected is in the correct format and location
            if len(epic_no) == 0:
                # In epic number store block_text part before name
                epic_no = block_text.split("Name")[0].strip()
                # Remove extra white space in between
                epic_no = " ".join(epic_no.split())
            
            parsed_data = parse_block_text(block_text, epic_no, counter)
            if parsed_data[1]:  # Only add if name is not empty
                combined_data.append(parsed_data)
                counter += 1

    df = pd.DataFrame(combined_data, columns=['S.No', 'Voter Full Name', 'Relative\'s Name', 'Relation Type', 'Age', 'Gender', 'House No', 'EPIC No'])
    return df

# List of images to process
image_paths = ['data/page_1.png', 'data/page_2.png']

# Process images and save combined results
df = process_multiple_images(image_paths)

df['Voter Full Name'] = df['Voter Full Name'].str.upper()
df['Relative\'s Name'] = df['Relative\'s Name'].str.upper()

# converting age from string to int and removing all enteries with age>88

def convert_to_int(s):
    try:
        return int(s)
    except ValueError:
        return None  # Or any other default value

df['Age'] = df['Age'].apply(convert_to_int)


df = df[df['Age'] <= 88]

# In EPIC No column, replacing starting 3 prefix with TXK if not present
df['EPIC No'] = df['EPIC No'].apply(lambda x: 'TXK'+x[3:] if x[:3]!='TXK' else x)

# Save to Excel
df.to_excel('Final_Output.xlsx', index=False)

print("Processing complete. Results saved to voter_data_combined.xlsx")


Processing complete. Results saved to voter_data_combined.xlsx
